# Rough Notebook to be changed to pct reduction
## Last Updated: 10-30-2020

In [593]:
# Importing the necessary libraries

import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
import json
import warnings
from os import listdir
from os.path import isfile,join
import datetime
import requests
from sklearn.metrics import r2_score, accuracy_score
import catboost
import matplotlib as mpl
import sys
sys.path.append('.')
from src.utils import collect_data
from src.utils import eda


In [594]:
import importlib
importlib.reload(collect_data)
collect_datasets = collect_data.DataCollection()
data_analysis = eda.DataAnalysis()

In [595]:
df_mobility = collect_datasets.get_mobility_data(download=False)
df_county_spending = collect_datasets.get_spend_data()
df_covid = collect_datasets.combine_covid_data()
df_weather = collect_datasets.combine_weather(df_mobility,df_covid)
df_county = collect_datasets.combined_county_data()

C:\Work\projects\mobility\US_Mobility_BTS\src\utils\collect_data.py:51: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_mobility['Wk #'] = (df_mobility.date.dt.week.astype(str) ) #+ ' ' + (df_mobility.date.dt.year.astype(str) )


In [596]:
def get_df_2019(df_weather, df_mob,df_covid,df_demographics,year=2019):
    df_mob = df_mob.dropna()
    thr_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
    if year==2020:
        df_mob_prep = pd.merge(left = df_mob, right = df_covid, on = ['fips', 'date'], how='inner')
        df_mob_prep = df_mob_prep[df_mob_prep.date <thr_date]
    else:
        df_mob_prep = df_mob[(df_mob.date < thr_date)]
        
    df_mob_weather = pd.merge(left = df_mob_prep, right = df_weather, on =['date', 'fips'])

    return df_mob_weather

In [597]:
df_2019 = get_df_2019(df_weather, df_mobility, df_covid, df_demographics=df_county)
df_2019_fil = df_2019[df_2019['State Postal Code'] != 'AK']

### Weather Correlation

In [604]:
df_2019_fips_corr = df_2019_fil.groupby('fips').corr()

In [623]:
df_2019_fips_corr_ = df_2019_fips_corr.reset_index(level=1)
cols = df_mobility.columns[7:] # ['trips_per_person','mobility_per_person','pct_stay_home','mobility_per_trip']
target_cols = [col for col in df_2019_fips_corr.columns[2:17] if col not in ['Number of Trips 50-100','Number of Trips 100-250',
                                                                        'Number of Trips 250-500','Number of Trips >=500']]

df_2019_fips_corr_temp = df_2019_fips_corr_[df_2019_fips_corr_.level_1=='FeelsLikeC'][target_cols].reset_index()

feelslike_tempC_corr_map = data_analysis.plot_map(target_cols,df_2019_fips_corr_temp,cmap='bwr',q_filter=False)

In [619]:
df_2019_fips_corr_ = df_2019_fips_corr.reset_index(level=1)
target_cols = [col for col in df_2019_fips_corr.columns[2:17] if col not in ['Number of Trips 50-100','Number of Trips 100-250',
                                                                        'Number of Trips 250-500','Number of Trips >=500']]

df_2019_fips_corr_sunH = df_2019_fips_corr_[df_2019_fips_corr_.level_1=='sunHour'][target_cols].reset_index()

sunhour_tempC_corr_map = data_analysis.plot_map(target_cols,df_2019_fips_corr_sunH,cmap='bwr',q_filter=True, filter_level=[0.005,0.995])